In [1]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import time
from threading import Thread

In [2]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)

In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = deque([], maxlen=self.capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def clear(self):
        self.memory.clear()

    def capacity(self):
        return self.capacity

    def __len__(self):
        return len(self.memory)

In [4]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [5]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
# PLOT_AVG is the number of episodes to average rewards over
# MAX_STEPS is the maximum number of steps to take in an episode
# VIDEO_FREQ is the frequency at which to record videos of the agent's performance
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
TAU = 0.005
LR = 1e-4
PLOT_AVG = 50
MAX_STEPS = 201
EPS_DECAY = MAX_STEPS * 20
VIDEO_FREQ = 10

In [6]:
def select_actions(envs, policy_net, states, rr):
    if rr == 0:
        with torch.no_grad():
            return policy_net(states).max(1).indices

    sample = random.random()

    if sample > rr:
      with torch.no_grad():
          return policy_net(states).max(1).indices
    else:
        return torch.tensor( envs.action_space.sample(), device=device, dtype=torch.long)

In [7]:
def plot_durations(episode_durations, show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())

    # Take PLOT_AVG episode averages and plot them too
    if len(durations_t) >= PLOT_AVG:
        means = durations_t.unfold(0, PLOT_AVG, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(PLOT_AVG - 1), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

def multi_plot_durations(data, show_results=False):
    num_datasets = len(data)
    num_columns = min(3, num_datasets)
    num_rows = (num_datasets + 2) // 3  # Calculate number of rows needed
    
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(6 * num_columns, 4 * num_rows), squeeze=False)

    for i, (ax, (label, d)) in enumerate(zip(axes.flat, data.items())):
        d = np.array(d)
        if show_results:
            ax.set_title("Results")
        else:
            ax.set_title('Training...')
        ax.set_xlabel('Episode')
        ax.set_ylabel(label)
        ax.plot(d, label=label)

        # Plot averages too
        window_size = min(PLOT_AVG, len(d))
        running_avg = np.convolve(d, np.ones(window_size)/window_size, mode='valid')
        x_values = np.arange(window_size - 1, len(d))
        ax.plot(x_values, running_avg, label='Running Average')

    plt.tight_layout()

    plt.pause(0.0001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_results:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [8]:
def optimize_model(policy_net, target_net, optimizer, memory):
    if len(memory) < BATCH_SIZE:
        return np.inf
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

    return loss

In [9]:
def do_step(envs, dones, policy_net, memory, states, steps_done, num_envs, n_observations, rr):
    actions = select_actions(envs, policy_net, states, rr)
    actions_cpu = actions.cpu().numpy()
    steps_done += 1

    observations, rewards, terminateds, truncateds, _ = envs.step(
        actions_cpu
    )

    heights = (np.sin(np.array(observations[:, 0]) - 1) + 0.75 ) / 0.25

    action_dir = np.array([-1, 0, 1]).take(actions_cpu)
    velocities = observations[:, 1]
    velocities = np.divide(velocities, np.absolute(velocities))  # converts to -1 or 1
    velocities = np.nan_to_num(velocities)  # converts nan to 0 (end of episode)
    #r = action_dir * velocities - 2

    r = heights - 2

    finish_reward = np.zeros(num_envs)
    finish_reward[terminateds] = MAX_STEPS
    r += finish_reward
    
    rewards = torch.tensor(r, device=device, dtype=torch.float32)

    # for all non-terminated states, set next_state
    next_states = torch.full(
        (num_envs, n_observations), torch.nan, device=device
    )
    obs = torch.tensor(observations, dtype=torch.float32, device=device)
    next_states[~terminateds] = obs[~terminateds]

    # Store the transition in memory
    for e in range(num_envs):
        if not dones[e]:
            memory.push(
                states[e].view(1, n_observations),
                actions[e].view(1, 1),
                (
                    next_states[e].view(1, n_observations)
                    if not terminateds[e]
                    else None
                ),
                rewards[e].view(1),
            )
        # if state not done
    # for e in num_envs

    dones = np.logical_or(dones, np.logical_or(terminateds, truncateds))

    # Move to the next state
    states = next_states

    return actions, dones, states, steps_done, heights, rewards

In [10]:
data = {"step_count": [], "max_height": [], "reward": [], "random_rate": [], "loss": [], "lr": []}
def train(num_envs = 4, num_episodes = 200, display_live = False):
    memory = ReplayMemory(5000)
    steps_done = 0

    envs = gym.make_vec("MountainCar-v0", num_envs=num_envs, vectorization_mode="async")
    record_env = gym.wrappers.RecordVideo(env=gym.make("MountainCar-v0", render_mode="rgb_array"), video_folder="mtn-videos/", name_prefix="train", disable_logger=True, episode_trigger=lambda x: True)

    # Get number of actions from gym action space
    n_actions = envs.single_action_space.n
    # Get the number of state observations
    n_observations = envs.single_observation_space.shape[0]
    
    policy_net = DQN(n_observations, n_actions).to(device)
    policy_net.train()
    target_net = DQN(n_observations, n_actions).to(device)
    target_net.load_state_dict(policy_net.state_dict())
    
    optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
    #schedule = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, verbose=True)
    schedule = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)

    for i_episode in range(num_episodes):

        # Initialize the environment and get its state
        states, infos = envs.reset()
        states = torch.tensor(
            states, dtype=torch.float32, device=device
        )  # .unsqueeze(0)

        dones = np.full(num_envs, False)
        max_heights = np.full(num_envs, -np.inf)
        avg_reward = 0
        min_step_count = MAX_STEPS

        if i_episode % VIDEO_FREQ == 0:
            record_done = False

            record_state, info = record_env.reset()
            record_state = torch.tensor(
                record_state, dtype=torch.float32, device=device
            ).unsqueeze(0)

            print("Recording video for episode", i_episode)

            policy_net.eval()
            while not record_done:
                with torch.no_grad():
                    action = policy_net(record_state).max(1).indices.item()
                record_state, _, record_termianted, record_truncated, _ = record_env.step(action)
                record_done = record_termianted or record_truncated
                record_state = torch.tensor(
                    record_state, dtype=torch.float32, device=device
                ).unsqueeze(0)
            # while environment not done
            policy_net.train()
        # if record episode

        schedule.step()

        for t in range(MAX_STEPS + 1):

            rr = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)

            actions, dones, states, steps_done, heights, rewards = do_step(
                envs, dones, policy_net, memory, states, steps_done, num_envs, n_observations, rr
            )
            max_heights = np.maximum(max_heights, heights)   
            avg_reward += rewards.mean().item()

            # Perform one step of the optimization (on the policy network)
            loss = optimize_model(policy_net, target_net, optimizer, memory)

            lr = schedule.get_last_lr()[0]

            # Soft update of the target network's weights
            # θ′ ← τ θ + (1 −τ )θ′
            target_net_state_dict = target_net.state_dict()
            policy_net_state_dict = policy_net.state_dict()
            for key in policy_net_state_dict:
                target_net_state_dict[key] = policy_net_state_dict[
                    key
                ] * TAU + target_net_state_dict[key] * (1 - TAU)
            target_net.load_state_dict(target_net_state_dict)

            if min_step_count == MAX_STEPS and dones.any():
                min_step_count = t

            if dones.all():  # once all are done
                data["max_height"].append(max_heights.mean())
                data["step_count"].append( (min_step_count + t) / 2 )
                data["reward"].append(avg_reward / t)
                data["random_rate"].append(rr)
                data["loss"].append(loss.item())
                data["lr"].append(lr)
                if display_live and i_episode % 5 == 0:
                    multi_plot_durations(data)
                break
            # if all done
        # for t steps

    # for i_episode

    envs.close()
    record_env.close()

    return data, target_net

In [11]:
def graphs_compare(*args, labels=None):
    if labels is None:
        labels = [f"Dataset {i+1}" for i in range(len(args))]
    
    num_datasets = len(args)
    num_columns = min(3, num_datasets)
    num_rows = (num_datasets + 2) // 3  # Calculate number of rows needed
    
    # Find global max and min across all datasets
    global_max = max(data.max() for data in args) + 10
    global_min = min(data.min() for data in args)

    global_x_max = max(data.shape[0] for data in args)
    
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(6 * num_columns, 4 * num_rows), squeeze=False)
    
    for i, (data, label) in enumerate(zip(args, labels)):
        ax = axes[i // num_columns, i % num_columns]  # Locate the correct subplot
        ax.plot(data)
        ax.set_title(label)
        ax.set_xlabel('Epochs')
        ax.grid(True)
        ax.set_ylim(global_min, global_max)  # Set the y-axis limit to be the same for all subplots
        ax.set_xlim(0, global_x_max)  # Set the x-axis limit to be the same for all subplots
    
    # Hide unused axes if any
    for j in range(i + 1, num_rows * num_columns):
        axes[j // num_columns, j % num_columns].axis('off')

    plt.tight_layout()
    plt.show()

In [12]:
data, target_net = train(8, 250, display_live=True)
multi_plot_durations(data, show_results=True)
plt.ioff()
plt.show()

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [ ]:
print('Complete')
multi_plot_durations(data, show_results=True)
plt.ioff()
plt.show()

In [ ]:
# VALIDATION
episodes = 200
env = gym.wrappers.RecordVideo(env=gym.make("MountainCar-v0", render_mode="rgb_array"), video_folder="mtn-videos/", name_prefix="val", disable_logger=True, episode_trigger=lambda x: x % 40 == 0)

data = {"step_count": []}
for i_episode in range(episodes):

    # Initialize the environment and get its state
    state, _ = env.reset()
    state = torch.tensor(
        state, dtype=torch.float32, device=device
    ).unsqueeze(0)

    for t in range(MAX_STEPS + 1):
        
        with torch.no_grad():
            action = target_net(state).max(1).indices.item()
        state, _, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        state = torch.tensor(
            state, dtype=torch.float32, device=device
        ).unsqueeze(0)

        if done:
            data["step_count"].append(t)
            if i_episode % 10 == 0:
              multi_plot_durations(data)
            break
        # if all done
    # for t steps

# for i_episode

multi_plot_durations(data, show_results=True)
plt.ioff()
plt.show()
print("Average: " + str(sum(data["step_count"]) / len(data["step_count"])))
print("Min: " + str(min(data["step_count"])))
print("Max: " + str(max(data["step_count"])))
print("Median: " + str(np.median(data["step_count"])))